In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Self explanatory stuff

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.exceptions import ConvergenceWarning
from warnings import simplefilter

dir_five = '/content/gdrive/MyDrive/EMG_Data/dataset_five'
dir_two = '/content/gdrive/MyDrive/EMG_Data/dataset_peace'
dir_one = '/content/gdrive/MyDrive/EMG_Data/dataset_one'

# These lists will contain all data samples (for example, f1c1 = gesture five, channel 1)
f1c1 = []
f1c2 = []
o1c1 = []
o1c2 = []
t1c1 = []
t1c2 = []
n1c1 = []
n1c2 = []

In [ ]:
# Reading each csv file and appending data samples to respective lists based on labels
# If label is 1, then data sample is appended to the list corresponding to csv file (one or two or five)
# If label is 0, then data sample is appended to the list of no gesture/rest gesture

for i in range(1,11):
    f1 = pd.read_csv(dir_five + '/five_'+str(i)+'_labeled.csv')
    o1 = pd.read_csv(dir_one + '/one_'+str(i)+'_labeled.csv')
    t1 = pd.read_csv(dir_two + '/peace_'+str(i)+'_labeled.csv')

    for i in range(len(f1['channel1_data'])):
        if f1['channel_1_upper_label'][i] == 1:
            f1c1.append(f1['channel1_data'][i])
            f1c2.append(f1['channel2_data'][i])
        else:
            n1c1.append(f1['channel1_data'][i])
            n1c2.append(f1['channel1_data'][i])

    for i in range(len(o1['channel1_data'])):
        if o1['channel_1_upper_label'][i] == 1:
            o1c1.append(o1['channel1_data'][i])
            o1c2.append(o1['channel2_data'][i])
        else:
            n1c1.append(o1['channel1_data'][i])
            n1c2.append(o1['channel1_data'][i])

    for i in range(len(t1['channel1_data'])):
        if t1['channel_1_upper_label'][i] == 1:
            t1c1.append(t1['channel1_data'][i])
            t1c2.append(t1['channel2_data'][i])
        else:
            n1c1.append(t1['channel1_data'][i])
            n1c2.append(t1['channel1_data'][i])

print("Datapoints for rest:", len(n1c1))
print("Datapoints for five:", len(f1c1))
print("Datapoints for two: ", len(t1c1))
print("Datapoints for one: ", len(o1c1))

Datapoints for rest: 1334056
Datapoints for five: 385423
Datapoints for two:  509324
Datapoints for one:  498466


In [ ]:
# Creating bins of 256 samples (this window is arbitrarily chosen; can be changed)

five1 = []
five2 = []
for i in range(0,385024,256):
    five1.append(f1c1[i:i+256])
    five2.append(f1c2[i:i+256])

two1 = []
two2 = []
for i in range(0,508928,256):
    two1.append(t1c1[i:i+256])
    two2.append(t1c2[i:i+256])

one1 = []
one2 = []
for i in range(0,497664,256):
    one1.append(o1c1[i:i+256])
    one2.append(o1c2[i:i+256])

rest1 = []
rest2 = []
for i in range(0,1331200,256):
    rest1.append(n1c1[i:i+256])
    rest2.append(n1c2[i:i+256])

print("Number of bins:", len(five1), len(two1), len(one1), len(rest1))

Number of bins: 1504 1988 1944 5200


In [ ]:
# Extracting time domain features from each bin

def power(x):
    p = 0
    for i in x:
        p = p + i*i
    return p

def rms(x):
    r = 0
    for i in x:
        r = r + i*i
    r = np.sqrt(r/len(x))
    return r

c1_amp, c2_amp = [], []
c1_slp, c2_slp = [], []
c1_pow, c2_pow = [], []
c1_rms, c2_rms = [], []
outp_label = []

for i in range(len(five1)):
    c1_amp.append(sum(five1[i]))
    c2_amp.append(sum(five2[i]))
    c1_pow.append(power(five1[i]))
    c2_pow.append(power(five2[i]))
    c1_rms.append(rms(five1[i]))
    c2_rms.append(rms(five2[i]))
    if i == 0:
        c1_slp.append(0)
        c2_slp.append(0)
    else:
        c1_slp.append(sum(five1[i])-sum(five1[i-1]))
        c2_slp.append(sum(five2[i])-sum(five2[i-1]))
    outp_label.append('five')

for i in range(len(two1)):
    c1_amp.append(sum(two1[i]))
    c2_amp.append(sum(two2[i]))
    c1_pow.append(power(two1[i]))
    c2_pow.append(power(two2[i]))
    c1_rms.append(rms(two1[i]))
    c2_rms.append(rms(two2[i]))
    if i == 0:
        c1_slp.append(0)
        c2_slp.append(0)
    else:
        c1_slp.append(sum(two1[i])-sum(two1[i-1]))
        c2_slp.append(sum(two2[i])-sum(two2[i-1]))
    outp_label.append('two')

for i in range(len(rest1)):
    c1_amp.append(sum(rest1[i]))
    c2_amp.append(sum(rest2[i]))
    c1_pow.append(power(rest1[i]))
    c2_pow.append(power(rest2[i]))
    c1_rms.append(rms(rest1[i]))
    c2_rms.append(rms(rest2[i]))
    if i == 0:
        c1_slp.append(0)
        c2_slp.append(0)
    else:
        c1_slp.append(sum(rest1[i])-sum(rest1[i-1]))
        c2_slp.append(sum(rest2[i])-sum(rest2[i-1]))
    outp_label.append('rest')

for i in range(len(one1)):
    c1_amp.append(sum(one1[i]))
    c2_amp.append(sum(one2[i]))
    c1_pow.append(power(one1[i]))
    c2_pow.append(power(one2[i]))
    c1_rms.append(rms(one1[i]))
    c2_rms.append(rms(one2[i]))
    if i == 0:
        c1_slp.append(0)
        c2_slp.append(0)
    else:
        c1_slp.append(sum(one1[i])-sum(one1[i-1]))
        c2_slp.append(sum(one2[i])-sum(one2[i-1]))
    outp_label.append('one')

In [ ]:
# To-do: Extracting frequency domain features from each bin


In [ ]:
# Preparing data for ML

X = np.array([c1_amp, c2_amp, c1_slp, c2_slp, c1_rms, c2_rms, c1_pow, c2_pow]).T
y = np.array(outp_label)

scaler = StandardScaler()
X = scaler.fit_transform(X)

print(np.shape(X), np.shape(y))

X_train, X_validate, y_train, y_validate = train_test_split(X, y, test_size=0.2, random_state=1) 

(10636, 8) (10636,)


In [ ]:
# Training Kernel SVM and Random Forest classifiers using GridSearchCV and displaying results 

param_grid_kersvm = {'C': [1, 10, 100], 
                    'gamma': [0.1, 0.01, 0.001],
                    'kernel': ['rbf']} 

param_grid_ranfor = {'max_depth': [4,8,12,16],
                    'max_features': [2,4,6,8]}

simplefilter("ignore", category=ConvergenceWarning)

print("Kernel SVM")
grid_kersvm = GridSearchCV(estimator = SVC(), param_grid = param_grid_kersvm, cv = 5)
grid_kersvm.fit(X_train, y_train)
y_predict = grid_kersvm.predict(X_validate)
print("balanced accuracy = ", balanced_accuracy_score(y_validate, y_predict))
print(grid_kersvm.best_params_)
print('Classification Report: \n', classification_report(y_validate,y_predict))
print('Confusion Matrix: \n', confusion_matrix(y_validate,y_predict))
print()

print("Random Forest")
grid_ranfor = GridSearchCV(estimator = RandomForestClassifier(), param_grid = param_grid_ranfor, cv = 5)
grid_ranfor.fit(X_train, y_train)
y_predict = grid_ranfor.predict(X_validate)
print("balanced accuracy = ", balanced_accuracy_score(y_validate, y_predict))
print(grid_ranfor.best_params_)
print('Classification Report: \n', classification_report(y_validate,y_predict))
print('Confusion Matrix: \n', confusion_matrix(y_validate,y_predict))
print()

Kernel SVM
balanced accuracy =  0.9083612471401574
{'C': 100, 'gamma': 0.1, 'kernel': 'rbf'}
Classification Report: 
               precision    recall  f1-score   support

        five       0.98      0.85      0.91       314
         one       0.86      0.93      0.89       362
        rest       0.94      1.00      0.97      1055
         two       1.00      0.86      0.92       397

    accuracy                           0.94      2128
   macro avg       0.94      0.91      0.92      2128
weighted avg       0.94      0.94      0.94      2128

Confusion Matrix: 
 [[ 268    7   39    0]
 [   1  335   25    1]
 [   2    0 1053    0]
 [   2   48    7  340]]

Random Forest
balanced accuracy =  0.9429172275096963
{'max_depth': 16, 'max_features': 6}
Classification Report: 
               precision    recall  f1-score   support

        five       0.98      0.93      0.95       314
         one       0.90      0.94      0.92       362
        rest       0.97      0.99      0.98      1055
